# Описание и цель исследования

Дана база данных сервиса, предоставляющего книги по подписке. В базе данных содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.
Следует ответить на вопросы: 
- Сколько книг вышло после 1 января 2000 года.
- Количество обзоров и среднюю оценку для каждой книги.
- Какое издательство, выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры)?
- Автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками).
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Используемые бибилиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

# Подключение к базе данных

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Таблицы 

In [3]:
query_books = ''' SELECT *
                FROM books
              '''

In [4]:
display(pd.io.sql.read_sql(query_books, con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [5]:
query_authors = '''  SELECT *
                     FROM authors
                '''

In [6]:
display(pd.io.sql.read_sql(query_authors, con = engine).head(5))

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query_ratings = ''' SELECT *
                    FROM ratings
                '''

In [8]:
display(pd.io.sql.read_sql(query_ratings, con = engine).head(5))

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query_publishers = ''' SELECT *
                       FROM publishers
                   '''

In [10]:
display(pd.io.sql.read_sql(query_publishers, con = engine).head(5))

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [11]:
query_reviews = ''' SELECT *
                    FROM reviews
                '''

In [12]:
display(pd.io.sql.read_sql(query_reviews, con = engine).head(5))

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Исследование данных

## Сколько книг вышло после 1 января 2000 года?

In [13]:
query = ''' SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
display(pd.io.sql.read_sql(query, con = engine).head(5))

,count
0,819


В базе данных содержится 819 книг, вышедших после 1 января 2000 года.

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [14]:
query = ''' SELECT b.book_id,
                   b.title,
                   COUNT(DISTINCT review_id) AS reviews_count,
                   (SUM(rat.rating) / COUNT(rat.rating)) AS mean_rating
            FROM reviews AS rev
            FULL OUTER JOIN books AS b on b.book_id = rev.book_id
            LEFT OUTER JOIN ratings AS rat on rev.book_id = rat.book_id
            GROUP BY b.book_id,
                     b.title
        '''
display(pd.io.sql.read_sql(query, con = engine))

,book_id,title,reviews_count,mean_rating
0,1,'Salem's Lot,2,3.0
1,2,1 000 Places to See Before You Die,1,2.0
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.0
3,4,1491: New Revelations of the Americas Before C...,2,4.0
4,5,1776,4,4.0
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.0
996,997,Xenocide (Ender's Saga #3),3,3.0
997,998,Year of Wonders,4,3.0
998,999,You Suck (A Love Story #2),2,4.0


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [15]:
query = ''' SELECT pub.publisher_id,
                   pub.publisher,
                   COUNT(b.book_id) AS books_count
            FROM books AS b
            LEFT OUTER JOIN publishers AS pub ON pub.publisher_id = b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY pub.publisher_id,
                     pub.publisher
            ORDER BY books_count DESC
            LIMIT 1
        '''
display(pd.io.sql.read_sql(query, con = engine))

,publisher_id,publisher,books_count
0,212,Penguin Books,42


Из содержащихся в базе данных издательств, наибольшее число книг (толще 50 страниц) было выпущено издательством Penguin Books. 

## Определите автора с самой высокой средней оценкой книг  (только книги с 50 и более оценками)

In [23]:
query = ''' SELECT authors.author_id,
                   authors.author,
                   CAST(SUM(ratings.rating) AS float) / CAST(COUNT(ratings.rating) AS float) AS mean_rating
            FROM books
            LEFT OUTER JOIN authors ON authors.author_id = books.author_id
            LEFT OUTER JOIN ratings ON books.book_id = ratings.book_id
            WHERE books.book_id IN (SELECT books.book_id
                                    FROM books
                                    LEFT OUTER JOIN ratings ON books.book_id = ratings.book_id
                                    GROUP BY books.book_id
                                    HAVING COUNT(ratings.rating_id) > 50)
            GROUP BY authors.author_id,
                     authors.author
            ORDER BY mean_rating DESC 
            LIMIT 1
        '''
display(pd.io.sql.read_sql(query, con = engine))

,author_id,author,mean_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


## - Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
query = ''' SELECT COUNT(reviews) / COUNT(DISTINCT username) AS mean_reviews
            FROM reviews
            WHERE reviews.username IN(SELECT username
                                      FROM ratings
                                      GROUP BY username
                                      HAVING COUNT(rating) > 50)
        '''
display(pd.io.sql.read_sql(query, con = engine))

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 1 обзор.